In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import time
import pandas as pd

### Daten

In [2]:
ort = "Magdeburg"
bundesland = "Sachsen-Anhalt"
immobilie = "Grundstueck"
art = "kaufen"

In [3]:
driver = webdriver.Firefox()

masterurl = 'https://www.immobilienscout24.de/Suche/de/' + str.lower(bundesland) + '/' + str.lower(ort) + '/' + str.lower(immobilie) + '-' + str.lower(art)

driver.get(masterurl)

time.sleep(15)

soup = BeautifulSoup(driver.page_source, 'html.parser')

listings = soup.find(id="resultlistpage")
pages = soup.find_all(class_="p-items")
teilbarab = soup.find_all(class_="font-tabular onlyLarge font-xs attribute-label")

active_page = soup.find(class_="p-items p-active")
active_page = [str(x) for x in active_page]
active_page = int(active_page[0][-5:-4])

last_page = pages[-2:-1]
last_page = [str(x) for x in last_page]
last_page = int(last_page[0][-10:-9])

In [4]:
df_final = pd.DataFrame()
df_final_0 = pd.DataFrame()

while active_page < last_page:

    priceqm = listings.find_all(attrs={"class":"font-highlight font-tabular"})
    location = listings.find_all(attrs={"class":"result-list-entry__map-link link-text-secondary font-normal font-ellipsis"})

    location = [str(x) for x in location]
    location = [x.replace('<button class="result-list-entry__map-link link-text-secondary font-normal font-ellipsis" data-result-id="', '') for x in location]
    location = [x.replace('" title="Auf der Karte anzeigen"><span class="s24-icons-s24_location_pin_1_24 result-list-entry__map-link__icon inline-block margin-right-xs"></span>', '') for x in location]
    location = [x.replace('</button>', '') for x in location]
    obj_id = [x[0:9] for x in location]
    location = [x[9:] for x in location]

    priceqm = [str(x) for x in priceqm]
    priceqm = [x.replace('<dd class="font-highlight font-tabular">', '') for x in priceqm]
    priceqm = [x.replace('</dd>', '') for x in priceqm]
    priceqm = [x.replace('<span><span class="desk-hide lap-hide">€ a. A.</span><span class="palm-hide">Kaufpreis</span></span>', '0') for x in priceqm]
    priceqm = [x.replace(' €', '')for x in priceqm]
    priceqm = [x.replace(' m²', '') for x in priceqm]
    priceqm = [x.replace('.', '')for x in priceqm]
    priceqm = [x.replace(',', '.')for x in priceqm]   

    result = []
    for x in priceqm:
        try:
            result.append(float(x))
        except:
            result.append(0.0)
    priceqm = result

    teilbarab = [str(x) for x in teilbarab]
    teilbarab = [x.replace('<dt class="font-tabular onlyLarge font-xs attribute-label">', '') for x in teilbarab]
    teilbarab = [x.replace('</dt>', '') for x in teilbarab]
    if "Teilbar ab" in teilbarab:
        index_teilbarab = [teilbarab.index("Teilbar ab")]

        for index in sorted(index_teilbarab, reverse=True):
            del priceqm[index]

    price = []
    area =[]
    price_per_qm = []

    for index in range(len(priceqm)):
        if index % 2 == 0:
            price.append(priceqm[index])
        else:
            area.append(priceqm[index])

    for index in range(len(price)):
        if price[index] != 0 and area[index] != 0:
            price_per_qm.append(price[index] / area[index])
        else:
            price_per_qm.append(0)

    data = {
        "Preis in €" : price,
        "Grundstücksfläche in m²" : area,
        "Preis/m²" : price_per_qm,
        "Ort" : location,
        "ImmoScout24 Objekt Id" : obj_id    
        }

    df = pd.DataFrame(data)

    df_sorted = df[df["Preis/m²"] != 0]
    df_sorted_0 = df[df["Preis/m²"] == 0]

    df_final = pd.concat([df_final, df_sorted])
    df_final_0 = pd.concat([df_final_0, df_sorted_0])

    active_page = soup.find(class_="p-items p-active")
    active_page = [str(x) for x in active_page]
    active_page = int(active_page[0][-5:-4])

    if active_page < last_page:

        url = masterurl + "?pagenumber=" + str(active_page + 1)

        driver.get(url)

        time.sleep(3)

        soup = BeautifulSoup(driver.page_source, 'html.parser')

        listings = soup.find(id="resultlistpage")
        pages = soup.find_all(class_="p-items")
        teilbarab = soup.find_all(class_="font-tabular onlyLarge font-xs attribute-label")

        url = masterurl

In [5]:
df_final = df_final.reset_index(drop=True)

In [6]:
df_final_filtered = df_final[df_final["Ort"].str.contains(ort)]
df_final_filtered

,Preis in €,Grundstücksfläche in m²,Preis/m²,Ort,ImmoScout24 Objekt Id
0,116380.0,529.0,220.000000,"Beyendorf-Sohlen, Magdeburg",147032891
1,184140.0,837.0,220.000000,"Beyendorf-Sohlen, Magdeburg",147032718
2,1180740.0,5367.0,220.000000,"Beyendorf-Sohlen, Magdeburg",147515781
3,273000.0,700.0,390.000000,"Nordwest, Magdeburg",140797793
4,199000.0,733.0,271.487040,"Diesdorf, Magdeburg",150700509
5,1470000.0,11206.0,131.179725,"Neue Neustadt, Magdeburg",153238507
6,1470000.0,11206.0,131.179725,"Neue Neustadt, Magdeburg",153238340
7,139000.0,510.0,272.549020,"Florian-Geyer-Straße 48, Sudenburg, Magdeburg",152194771
8,155000.0,600.0,258.333333,"Neue Neustadt, Magdeburg",153152035
9,205000.0,900.0,227.777778,"Cracau, Magdeburg",153152037


In [7]:
df_final_filtered.describe()

,Preis in €,Grundstücksfläche in m²,Preis/m²
count,4.500000e+01,45.000000,45.000000
mean,3.976600e+05,2019.666667,332.624827
std,4.503977e+05,3564.841989,331.491881
min,8.500000e+03,180.000000,5.017710
25%,1.593000e+05,600.000000,217.050298
50%,2.200000e+05,850.000000,272.549020
75%,3.000000e+05,1341.000000,337.982833
max,1.881490e+06,19005.000000,1962.323391


In [8]:
df_final_0 = df_final_0.reset_index(drop=True)

In [9]:
df_final_filtered_0 = df_final_0[df_final_0["Ort"].str.contains(ort)]
df_final_filtered_0

,Preis in €,Grundstücksfläche in m²,Preis/m²,Ort,ImmoScout24 Objekt Id
0,0.0,429.00,0.0,"Neue Neustadt, Magdeburg",141622336
1,0.0,600.00,0.0,"Neue Neustadt, Magdeburg",153152033
2,114898.0,0.00,0.0,"Beyendorf-Sohlen, Magdeburg",146235295
3,0.0,4901.00,0.0,"Rothenseer Straße 68, Neue Neustadt, Magdeburg",150140263
4,0.0,5810.00,0.0,"Herrenkrug, Magdeburg",150592731
5,0.0,770.00,0.0,"Kümmelsberg, Stadtfeld West, Magdeburg",147203512
6,0.0,858.25,0.0,"Neue Neustadt, Magdeburg",145224236
7,0.0,685.25,0.0,"Neue Neustadt, Magdeburg",145223650
8,0.0,663.00,0.0,"Lemsdorf, Magdeburg",134395487
9,0.0,575.00,0.0,"Lemsdorf, Magdeburg",134394292


In [10]:
driver.quit()